## Properties

Uma das características principais de orientação a objetos é que a representação interna de dados de um tipo fica *encapsulada* através do uso dos métodos (operações) associados com esse tipo. A vantagem é que características do tipo podem ser calculadas quando necessárias, ou pode-se realizar a mudança de estado do objeto de forma controlada (por exemplo, verificando novos valores).

Para mostrar essa vantagem, vamos supor que você precisa de uma classe cujos objetos irão armazenar valores inteiros positivos. Uma forma simples de fazer isso, sem encapsulação, seria como na classe abaixo.

In [1]:
class PositivoRuim:
    def __init__(self, val):
        self.valor = val

Agora podemos criar um objeto com um valor inicial desejado.

In [2]:
pr = PositivoRuim(3)

E é bastante simples de ler ou alterar o valor associado:

In [3]:
pr.valor

3

In [4]:
pr.valor += 3

In [5]:
pr.valor

6

No entanto, essa implementação permite que o objeto seja alterado para um valor inválido:

In [6]:
pr.valor = -2

In [7]:
pr.valor

-2

Para resolver esse problema, devemos usar encapsulação, fornecendo métodos para a leitura e alateração do valor:

In [8]:
class PositivoV1:
    def __init__(self, ini):
        if ini <= 0:
            raise ValueError('Precisa ser positivo')
        self.__val = ini
    def le_valor(self):
        return self.__val
    def muda_valor(self, x):
        if x <= 0:
            raise ValueError('Precisa ser positivo')
        self.__val = x

Agora sabemos que o objeto nunca terá valores negativos ou zero.

In [9]:
pv = PositivoV1(3)

In [10]:
pv.le_valor()

3

In [11]:
pv.muda_valor(5)

In [12]:
pv.le_valor()

5

In [13]:
pv.muda_valor(-2)

ValueError: Precisa ser positivo

Infelizmente, o uso do objeto fica desagradável:

In [14]:
pv.muda_valor(pv.le_valor() + 2) # pv += 2
pv.le_valor()

7

Para evitar esse tipo de problema, Python permite que você implemente métodos para acessar o objeto e os use como se você estivesse acessando diretamente um atributo do objeto:

In [15]:
class Positivo:
    def __init__(self, ini):
        if ini <= 0:
            raise ValueError('Precisa ser positivo')
        self.__val = ini
    def le_valor(self):
        return self.__val
    def muda_valor(self, x):
        if x <= 0:
            raise ValueError('Precisa ser positivo')
        self.__val = x
    valor = property(le_valor, muda_valor, None, None)

O código da última linha da classe acima cria um atributo denominado `valor` que, quando lido, provoca uma chamada para o método `le_valor` e quando alterado provoca uma chama para o método `muda_valor`. Os dois `None` no final da chamada são locais onde se pode colocar uma função a ser chamada quando se faz `del` no atributo criado e uma *docstringc* para o atributo.

In [16]:
p = Positivo(4)

In [17]:
p.valor

4

In [18]:
p.valor = 5

In [19]:
p.valor

5

In [20]:
p.valor = -1

ValueError: Precisa ser positivo

Isso inclusive funciona quando o acesso é feito indiretamente por outros métodos, como pelo método `__iadd__` no código abaixo.

In [21]:
p.valor += -6

ValueError: Precisa ser positivo

## Métodos estáticos

Já vimos anteriormente que podemos definir atributos associados à classe (isto é, ao objeto que representa a classe).

Da mesma forma, podemos definir métodos que se associam com a classe, ao invés de com os objetos dessa classe. Esses são denominados *métodos estáticos* ou *métodos de classe*.

Obviamente, esses métodos não podem acessar atributos dos objetos da classe, mas apenas atributos da classe.

No exemplo abaixo, `x` é um atributo da classe (criado na definição da classe), quanto `y` é um atributo dos objetos da classe (criado quando o objeto é criado e `__init__` executado).

O método `normal` é um método associado aos objetos, como costumamos usar. Já os métodos `estatico` e `de_classe` são associados à classe. A diferença entre métodos estáticos e de classe é que métodos estáticos recebem apenas os parâmetros que são explicitamente passados a eles, enquanto que os de classe recebem a classe do objeto sobre o qual foram chamados. Veja o código abaixo.

In [22]:
class A:
    x = 0
    def __init__(self, y):
        self.y = y
    def normal(self):
        return self, self.x, self.y
    def estatico():
        return A, A.x
    def de_classe(cls, z):
        return cls, cls.x + z
    estatico = staticmethod(estatico)
    de_classe = classmethod(de_classe)

As chamadas a `staticmethod` e `classmethod` são necessárias para definir corretamente esses métodos. Veja exemplos de uso abaixo.

In [23]:
a = A(2)

O objeto `a` é da classe `A`, então podemos chamar métodos normais sobre ele. Note como o método normal recebe em `self` o objeto sobre o qual a chamada foi feita, e tem acesso tanto ao atributo próprio `y` como ao atributo da classe `x`.

In [24]:
a.normal()

(<__main__.A at 0x7f9b28e7ccf8>, 0, 2)

In [ ]:
a

Já o método `estatico` não recebe nenhum parâmetro (apesar de ter sido chamado no formato `a.estatico()`) e tem acesso apenas ao atributo `x` da classe.

In [25]:
a.estatico()

(__main__.A, 0)

O método `de_classe`, por outro lado, ao ser chamado como `a.de_classe(4)`, recebe como primeiro parâmetro a classe do objeto à esquerda do ponto:

In [26]:
a.de_classe(4)

(__main__.A, 4)

Para entender melhor a diferença entre métodos estáticos e de classe, veja o que ocorre em caso de herança. Note como o método de classe tem acesso à classe exata do objeto, mesmo que ele seja chamado para objeto de classe derivada.

In [27]:
class B(A):
    pass

In [28]:
b = B(5)

In [29]:
b.normal()

(<__main__.B at 0x7f9b24240860>, 0, 5)

In [30]:
b.estatico()

(__main__.A, 0)

In [31]:
b.de_classe(8)

(__main__.B, 8)

## Decoradores

Em Python, decoradores são elementos que podem ser usados para alterar o comportamento de funções (decoradores de funções) ou classes (decoradores de classes) sobre os quais são aplicados.

Se definimos um decorador denominado `decor` (por exemplo), para aplicá-lo basta usar a sintaxe `@decor` imediatamente antes da função ou classe que será decorada.

A linguagem já define diversos decoradores. Alguns importantes são ligados a propriedades e métodos estáticos e de classe.

Por exemplo, podemos definir os métodos estáticos e de classe do exemplo anterior de forma mais simples como na classe seguinte:

In [32]:
class A:
    x = 0
    def __init__(self, y):
        self.y = y
    def normal(self):
        return self, self.x, self.y
    @staticmethod
    def estatico():
        return A, A.x
    @classmethod
    def de_classe(cls, z):
        return cls, cls.x + z

In [33]:
a = A(2)

In [34]:
a.normal()

(<__main__.A at 0x7f9b2424cb38>, 0, 2)

In [35]:
a.estatico()

(__main__.A, 0)

In [36]:
a.de_classe(3)

(__main__.A, 3)

Existe também o decorador `property`, usado para definir uma *property* de forma mais simples: Basta colocar esse decorador antes da função que será usada para acessos de leitura. O nome da função será o nome do atributo criado. Depois, se você quiser controlar também a escrita, basta usar o decorador `nome_do_atributo.setter` antes da função que será usada para a escrita (e que deve ter o mesmo nome do atributo). Veja abaixo redefinição da classe `Positivo`.

In [37]:
class Positivo:
    def __init__(self, ini):
        if ini <= 0:
            raise ValueError('Precisa ser positivo')
        self.__val = ini
    @property
    def valor(self):
        return self.__val
    @valor.setter
    def valor(self, x):
        if x <= 0:
            raise ValueError('Precisa ser positivo')
        self.__val = x

In [38]:
p4 = Positivo(1)

In [39]:
p4.valor

1

In [40]:
p4.valor = 5

In [41]:
p4.valor

5

In [42]:
p4.valor += 2

In [43]:
p4.valor -= 10

ValueError: Precisa ser positivo

## Definindo seus decoradores

Você também pode definir seus próprios decoradores. 

Para definir um decorador de funções você deve criar um objeto que define o método `__call__`, isto é, ele pode ser chamado como uma função. Esse método irá fazer o processamento necessário no lugar da função que foi decorada.

Para entender o processo, você deve pensar que o código

    @decor
    def f(a, b):
        ...

será traduzido para:

    def f(a, b):
        ...
    f = decor(f)

Portanto o seu decorador deve aceitar como parâmetro a função a ser decorada e retornar uma nova função que pode ser utilizada no lugar dela.

Abaixo definimos a classe `tracer` que pode ser usada como um decorador de funções e, para cada chamada da função, imprime o número de vezes que ela já foi chamada, antes de efetivamente realizar a chamada.

In [44]:
class tracer:
    def __init__(self, fun):
        self.calls = 0
        self.fun = fun
    def __call__(self, *args, **kargs):
        self.calls += 1
        print('chamada numero %d da função %s' % (self.calls, self.fun.__name__))
        return self.fun(*args, **kargs)

Veja como no `__init__` recebemos uma função (`fun`) que é guardada no objeto (`self.fun`). Depois, quando o objeto criado for chamado (`__call__`) pegamos os argumentos passados e os passamos para `self.fun` (no `return`), mas antes disso incrementamos e mostramos o valor do número de chamadas (`self.calls`).

O uso do decorador é simples:

In [45]:
@tracer
def f(x, y):
    print('Em f')
    return x + 2*y

In [46]:
f(2,4)

chamada numero 1 da função f
Em f


10

In [47]:
f(5,6)

chamada numero 2 da função f
Em f


17

In [48]:
f(0,-1); f(5,2); f(10,15)

chamada numero 3 da função f
Em f
chamada numero 4 da função f
Em f
chamada numero 5 da função f
Em f


40

O decorador definido acima tem um defeito: ele não serve para decorar métodos de classes. Veja o que acontece no código abaixo:

In [49]:
class B:
    @tracer
    def f(self):
        return self

In [50]:
b = B()

In [51]:
b.f()

chamada numero 1 da função f


TypeError: f() missing 1 required positional argument: 'self'

O problema é que o atributo `self` incluido nas chamadas de métodos não está sendo corretamente passado. Não adianta passar o `self` recebido pelo `__call__`, pois este será o objeto do tipo `tracer`, ao invés de ser o objeto desejado do tipo `B`.

A solução é usar uma *closure*, como no código abaixo.

In [52]:
def mtracer(fun):
    def oncall(*args, **kargs):
        oncall.calls += 1
        print('chamada numero %d da função %s' % (oncall.calls, fun.__name__))
        return fun(*args, **kargs)
    oncall.calls = 0
    return oncall

Note como aqui o decorador é uma função (e não uma classe), que recebe a função original como parâmetro `fun`. Ele então cria uma nova função, denominada `oncall`, definida internamente (uma *closure*) que faz acesso a `fun` na hora que for chamada. Essa é a função que será chamada, executando o incremento do seu atributo `calls` sempre que for chamada. (Como funções são também objetos, elas podem ter atributos.)

In [53]:
@mtracer
def g(x, y):
    print('Em f')
    return x ** y - 1

O código acima será traduzido em

    def g(x, y):
        print('Em f')
        return x ** y - 1
    g = mtracer(g)

E portanto `g` será substituida pela função `oncall` que faz acesso a `g` (guardado em `oncall.fun`) em seu `return`.

In [54]:
g(2,4)

chamada numero 1 da função g
Em f


15

In [55]:
g(3,5)

chamada numero 2 da função g
Em f


242

Esse decorador funciona também para métodos, pois não há um objeto adicional no meio do caminho (o método da classe `B` abaixo é substituido pelo decorador).

In [56]:
class B:
    @mtracer
    def f(self):
        return self

In [57]:
b = B()

In [58]:
b.f()

chamada numero 1 da função f


In [59]:
b.f()

chamada numero 2 da função f


Agora vejamos um exemplo de um decorador de classes.

Para entender decoradores de classes, você precisa se lembrar que um código como:

    @decor
    class A:
        ...

será substituido por:

    class A:
        ...
    A = decor(A)

Portanto, o seu decorador deve retornar uma *classe*, cujos objetos servirão como substitutos para os objetos da classe `A`.

No exemplo abaixo, definimos um decorador que altera a forma de converter objetos da classe decorada para cadeias de caracteres (acrescentando uma mensagem).

In [60]:
def existencialista(cls):
    class Ex(cls):
        def __init__(self, *args, **kargs):
            cls.__init__(self, *args, **kargs)
        def __repr__(self):
            return cls.__repr__(self) + '[Mas a vida é inútil!]'
    return Ex

Veja como o decorador é uma função que define internamente uma classe e a retorna. Essa classe interna faz referência à classe original (armazenada como `cls`) e a usa para sua implementação.

In [61]:
@existencialista
class S1:
    def __init__(self, x):
        self.val = x
    def __repr__(self):
        return 'S1(' + str(self.val) + ')'

In [62]:
s1 = S1(10)

In [63]:
s1

S1(10)[Mas a vida é inútil!]

In [64]:
@existencialista
class minhalista(list):
    pass

In [65]:
x = minhalista()
x.append(1); x.append(3); x.append(7)

In [66]:
x

[1, 3, 7][Mas a vida é inútil!]